# Introduction

#### Our client Lily is travelling to Manhattan the next month, and she will spend about 2 weeks there. She wants to enjoy her stay with nice restaurants and great hotels
#### Lily wants to stay in 2 different hotels to experience different lifestyle of the nearby neighborhoods in the two weeks

-----------------------------------------------------

# Data

#### We will use the Manhattan neighborhood data from the lab( https://geo.nyu.edu/catalog/nyu_2451_34572) which contains the Borough, the name and latittude, longitude of each neighborhood in Manhattan
#### And we will also utilize the Foursquare API, to get the location information for these neighborhoods
#### Then we need to group these neighborhoods into clusters (for example: 4-5 clusters), so that we can have different clusters of neighborhoods that are similar within the cluster yet different from other clusters
#### In this way, by choosing neighborhoods from diferent clusters, the experience of lifestyle would be different
#### Review the cluster characteristics and find two neighborhoods form the most suitable clusters that have many restaurants and recommend a good hotel from each cluster for Lily.Introduction



In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Libraries imported.


##### Tranform the data into a pandas dataframe

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
neighborhoods.head()

Data downloaded!


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [3]:
neighborhoods.shape

(306, 4)

##### Neighborhoods in Manhattan

In [4]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [5]:
manhattan_data.shape
print('There are  {} neighborhoods in Manhattan in total.' .format(manhattan_data.shape[0]))

There are  40 neighborhoods in Manhattan in total.




Use geopy library to get the latitude and longitude values of Manhattan

In [6]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


In [7]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

#### now we have the neighborhood data(name,Latitude,Longitude of the neighborhood in Manhattan), next we will utilize the foursquare API to further explore/search the area

##### Explore Neighborhoods in Manhattan

In [8]:
# The code was removed by Watson Studio for sharing.

In [9]:
radius=500
LIMIT=100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        #print(results)
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['id'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue id',           
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


## Now we already have the venue information for each neighborhood in Manhattan, we can explore and research more in the following steps!~

In [11]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3324, 8)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue id,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,4b4429abf964a52037f225e3,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,4baf59e8f964a520a6f93be3,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,4b79cc46f964a520c5122fe3,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin' Donuts,4b5357adf964a520319827e3,40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Starbucks,55f81cd2498ee903149fcc64,40.877531,-73.905582,Coffee Shop


In [12]:
#manhattan_venues['Venue Category'].unique()
len(manhattan_venues['Venue Category'].unique())


328

In [13]:
#check how many restaurants each neighborhood has
restaurant_count=manhattan_venues[manhattan_venues['Venue Category'].str.contains("Restaurant")].groupby('Neighborhood').count()[['Venue']]
restaurant_count.rename(columns={"Venue": "restaurant_count"},inplace=True)
restaurant_count.reset_index(inplace=True)
restaurant_count

,Neighborhood,restaurant_count
0,Battery Park City,9
1,Carnegie Hill,23
2,Central Harlem,14
3,Chelsea,26
4,Chinatown,43
5,Civic Center,23
6,Clinton,22
7,East Harlem,16
8,East Village,38
9,Financial District,22


In [14]:
#check how many hotel each neighborhood has
Hotel_count=manhattan_venues[manhattan_venues['Venue Category']=="Hotel"].groupby('Neighborhood').count()[['Venue']]
Hotel_count.rename(columns={"Venue": "hotel_count"},inplace=True)
Hotel_count.reset_index(inplace=True)
Hotel_count

,Neighborhood,hotel_count
0,Battery Park City,5
1,Chelsea,3
2,Chinatown,1
3,Civic Center,2
4,Clinton,4
5,Financial District,5
6,Gramercy,2
7,Greenwich Village,1
8,Hudson Yards,4
9,Lincoln Square,2


The neighborhood must have at least one hotel

In [15]:
manhattan_venues=pd.merge(manhattan_venues, Hotel_count[['Neighborhood']], how='inner', on='Neighborhood')
manhattan_venues.shape

(2056, 8)

In [16]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,American Restaurant,Animal Shelter,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Theater,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Resort,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skate Park,Ski Shop,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Toy / Game Store,Trail,Tree,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Volleyball Court,Watch Shop,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Chinatown,0,0,0,0,0,0,0,

In [17]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,American Restaurant,Animal Shelter,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Theater,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Resort,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skate Park,Ski Shop,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Toy / Game Store,Trail,Tree,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Volleyball Court,Watch Shop,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Battery Park City,0.00,0

In [18]:
restaurants=manhattan_grouped.columns
#manhattan_grouped['res_score']=manhattan_grouped[restaurants[restaurants.str.contains('Restaurant')]].sum(axis=1)

manhattan_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,American Restaurant,Animal Shelter,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Theater,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Resort,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skate Park,Ski Shop,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Toy / Game Store,Trail,Tree,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Volleyball Court,Watch Shop,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Battery Park City,0.0,0.

In [19]:
manhattan_grouped.shape

(21, 271)

find the top 10 most common venues for each neighborhood

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Hotel,Gym,Shopping Mall,Wine Shop,Italian Restaurant,Cupcake Shop,Food Truck,Fountain
1,Chelsea,Coffee Shop,Ice Cream Shop,Italian Restaurant,American Restaurant,Bakery,Nightclub,Theater,Hotel,Seafood Restaurant,Bookstore
2,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Dim Sum Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Noodle House,Salon / Barbershop,Hotpot Restaurant,Bar
3,Civic Center,Italian Restaurant,Gym / Fitness Center,Bakery,French Restaurant,Yoga Studio,Coffee Shop,Sporting Goods Shop,Spa,Sandwich Place,Cocktail Bar
4,Clinton,Theater,Hotel,American Restaurant,Gym / Fitness Center,Italian Restaurant,Wine Shop,Gym,Spa,Coffee Shop,Lounge


#### Cluster neighborhoods

In [22]:
# set number of clusters
kclusters = 6

manhattan_grouped_clustering = manhattan_grouped.drop(['Neighborhood'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 3, 0, 2, 1, 0, 0, 2, 2], dtype=int32)

In [23]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [24]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Battery Park City,Park,Coffee Shop,Hotel,Gym,Shopping Mall,Wine Shop,Italian Restaurant,Cupcake Shop,Food Truck,Fountain
1,0,Chelsea,Coffee Shop,Ice Cream Shop,Italian Restaurant,American Restaurant,Bakery,Nightclub,Theater,Hotel,Seafood Restaurant,Bookstore
2,3,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Dim Sum Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Noodle House,Salon / Barbershop,Hotpot Restaurant,Bar
3,0,Civic Center,Italian Restaurant,Gym / Fitness Center,Bakery,French Restaurant,Yoga Studio,Coffee Shop,Sporting Goods Shop,Spa,Sandwich Place,Cocktail Bar
4,2,Clinton,Theater,Hotel,American Restaurant,Gym / Fitness Center,Italian Restaurant,Wine Shop,Gym,Spa,Coffee Shop,Lounge


In [25]:
#merge certain tables to include useful information for each neighborhood

manhattan_merged=manhattan_data
#manhattan_merged = pd.merge(manhattan_merged,manhattan_data[['Borough','Neighborhood','Latitude','Longitude']],on='Neighborhood')

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood',how='right')
#manhattan_merged = pd.merge(neighborhoods_venues_sorted,manhattan_data[['Borough','Neighborhood','Latitude','Longitude']],on='Neighborhood',how='left')

manhattan_merged = manhattan_merged.join(restaurant_count.set_index('Neighborhood'),on='Neighborhood',how='left')
manhattan_merged = manhattan_merged.join(Hotel_count.set_index('Neighborhood'),on='Neighborhood',how='left')
manhattan_merged.head() # check the last columns!
#manhattan_merged.shape

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,restaurant_count,hotel_count
1,Manhattan,Chinatown,40.715618,-73.994279,3,Chinese Restaurant,Cocktail Bar,American Restaurant,Dim Sum Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Noodle House,Salon / Barbershop,Hotpot Restaurant,Bar,43,1
8,Manhattan,Upper East Side,40.775639,-73.960508,0,Italian Restaurant,Exhibit,Coffee Shop,Juice Bar,Art Gallery,Bakery,Gym / Fitness Center,Cocktail Bar,Spa,Hotel,24,3
13,Manhattan,Lincoln Square,40.773529,-73.985338,2,Theater,Café,Concert Hall,Plaza,Italian Restaurant,Gym / Fitness Center,Opera House,French Restaurant,Indie Movie Theater,Performing Arts Venue,16,2
14,Manhattan,Clinton,40.759101,-73.996119,2,Theater,Hotel,American Restaurant,Gym / Fitness Center,Italian Restaurant,Wine Shop,Gym,Spa,Coffee Shop,Lounge,22,4
15,Manhattan,Midtown,40.754691,-73.981669,2,Hotel,Clothing Store,Cocktail Bar,Steakhouse,Theater,American Restaurant,Bakery,Bookstore,Sporting Goods Shop,Spa,20,8


In [26]:
manhattan_merged.shape

(21, 17)

In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [28]:
print('The average restaurant count in all the neighborhoods is' ,manhattan_merged['restaurant_count'].mean())

The average restaurant count in all the neighborhoods is 26.666666666666668


#### Let's look at the cluster means can review the average number of restaurants and hotels in each cluster

In [29]:
manhattan_merged.groupby('Cluster Labels').mean()

,Latitude,Longitude,restaurant_count,hotel_count
Cluster Labels,,,,
0,40.741361,-73.985081,30.000000,2.375
1,40.709520,-74.013767,15.500000,5.000
2,40.752066,-73.987550,24.142857,4.000
3,40.715618,-73.994279,43.000000,1.000
4,40.748510,-73.988713,31.000000,6.000
5,40.720754,-73.998981,23.000000,1.500


#### looks like the cluster 1,3 and 4 have the highest average number of restaurants (higher than the average), based on our client need, we will choose neighborhoods from these clusters

#### Next, let check how many neighborhoods are included in each cluster, and we noticed that there're 2,2 and 1 neighborhoods within each cluster respectively.

In [30]:
manhattan_merged.groupby('Cluster Labels').count()[['Neighborhood']]

,Neighborhood
Cluster Labels,
0,8
1,2
2,7
3,1
4,1
5,2


In [31]:
clu_no = [1,3,4]

options=manhattan_merged[manhattan_merged['Cluster Labels'].isin(clu_no)]
options.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,restaurant_count,hotel_count
1,Manhattan,Chinatown,40.715618,-73.994279,3,Chinese Restaurant,Cocktail Bar,American Restaurant,Dim Sum Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Noodle House,Salon / Barbershop,Hotpot Restaurant,Bar,43,1
28,Manhattan,Battery Park City,40.711932,-74.016869,1,Park,Coffee Shop,Hotel,Gym,Shopping Mall,Wine Shop,Italian Restaurant,Cupcake Shop,Food Truck,Fountain,9,5
29,Manhattan,Financial District,40.707107,-74.010665,1,Coffee Shop,Hotel,Gym,Wine Shop,Steakhouse,Pizza Place,Park,Bar,Food Truck,Italian Restaurant,22,5
33,Manhattan,Midtown South,40.748510,-73.988713,4,Korean Restaurant,Hotel,Hotel Bar,Japanese Restaurant,Coffee Shop,Cosmetics Shop,Italian Restaurant,Bakery,Boutique,Cocktail Bar,31,6


In [32]:
# create map with only cluster 1, 3 and 4
map_clusters2 = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(3)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 2, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(options['Latitude'], options['Longitude'], options['Neighborhood'], options['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters2)
       
map_clusters2

#### let's look at the cluster 1, 3 and 4 in more details

In [33]:
options.loc[options['Cluster Labels'] == 1, options.columns[[1] + list(range(5, options.shape[1]))]]
options.loc[options['Cluster Labels'] == 1]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,restaurant_count,hotel_count
28,Manhattan,Battery Park City,40.711932,-74.016869,1,Park,Coffee Shop,Hotel,Gym,Shopping Mall,Wine Shop,Italian Restaurant,Cupcake Shop,Food Truck,Fountain,9,5
29,Manhattan,Financial District,40.707107,-74.010665,1,Coffee Shop,Hotel,Gym,Wine Shop,Steakhouse,Pizza Place,Park,Bar,Food Truck,Italian Restaurant,22,5


In [34]:
options.loc[options['Cluster Labels'] == 3]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,restaurant_count,hotel_count
1,Manhattan,Chinatown,40.715618,-73.994279,3,Chinese Restaurant,Cocktail Bar,American Restaurant,Dim Sum Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Noodle House,Salon / Barbershop,Hotpot Restaurant,Bar,43,1


In [35]:
options.loc[options['Cluster Labels'] == 4]

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,restaurant_count,hotel_count
33,Manhattan,Midtown South,40.74851,-73.988713,4,Korean Restaurant,Hotel,Hotel Bar,Japanese Restaurant,Coffee Shop,Cosmetics Shop,Italian Restaurant,Bakery,Boutique,Cocktail Bar,31,6


In [36]:
options2=options[options['Neighborhood']!='Soho']
options2

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,restaurant_count,hotel_count
1,Manhattan,Chinatown,40.715618,-73.994279,3,Chinese Restaurant,Cocktail Bar,American Restaurant,Dim Sum Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Noodle House,Salon / Barbershop,Hotpot Restaurant,Bar,43,1
28,Manhattan,Battery Park City,40.711932,-74.016869,1,Park,Coffee Shop,Hotel,Gym,Shopping Mall,Wine Shop,Italian Restaurant,Cupcake Shop,Food Truck,Fountain,9,5
29,Manhattan,Financial District,40.707107,-74.010665,1,Coffee Shop,Hotel,Gym,Wine Shop,Steakhouse,Pizza Place,Park,Bar,Food Truck,Italian Restaurant,22,5
33,Manhattan,Midtown South,40.748510,-73.988713,4,Korean Restaurant,Hotel,Hotel Bar,Japanese Restaurant,Coffee Shop,Cosmetics Shop,Italian Restaurant,Bakery,Boutique,Cocktail Bar,31,6


In [37]:
Hotel_list=manhattan_venues[manhattan_venues['Venue Category']=="Hotel"]

In [38]:
Hotel_list.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue id,Venue Latitude,Venue Longitude,Venue Category
6,Chinatown,40.715618,-73.994279,Hotel 50 Bowery NYC,578692f4498e1054905dbde7,40.715936,-73.996789,Hotel
106,Upper East Side,40.775639,-73.960508,The Mark Hotel,4b37853af964a520c54125e3,40.775201,-73.963351,Hotel
111,Upper East Side,40.775639,-73.960508,The Carlyle,4ac8d0d3f964a520b3bc20e3,40.774413,-73.963301,Hotel
137,Upper East Side,40.775639,-73.960508,The Surrey,4ae0e432f964a5206e8321e3,40.774415,-73.963889,Hotel
227,Lincoln Square,40.773529,-73.985338,The Phillips Club,4b515b87f964a520a84b27e3,40.774415,-73.982925,Hotel


In [39]:
manhattan_merged.groupby('Cluster Labels').sum()

,Latitude,Longitude,restaurant_count,hotel_count
Cluster Labels,,,,
0,325.930887,-591.880651,240,19
1,81.419039,-148.027535,31,10
2,285.264461,-517.912848,169,28
3,40.715618,-73.994279,43,1
4,40.748510,-73.988713,31,6
5,81.441508,-147.997961,46,3


In [40]:
cls_hotels=Hotel_list.join(options2.set_index('Neighborhood'),on='Neighborhood',how='right').rename(columns={'Venue':'Hotel'})

In [41]:
cls_hotels.set_index('Venue id')

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Hotel,Venue Latitude,Venue Longitude,Venue Category,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,restaurant_count,hotel_count
Venue id,,,,,,,,,,,,,,,,,,,,,,,
578692f4498e1054905dbde7,Chinatown,40.715618,-73.994279,Hotel 50 Bowery NYC,40.715936,-73.996789,Hotel,Manhattan,40.715618,-73.994279,3,Chinese Restaurant,Cocktail Bar,American Restaurant,Dim Sum Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Noodle House,Salon / Barbershop,Hotpot Restaurant,Bar,43,1
4f22ca77e4b0ed3396a83a05,Battery Park City,40.711932,-74.016869,Conrad New York,40.714911,-74.015461,Hotel,Manhattan,40.711932,-74.016869,1,Park,Coffee Shop,Hotel,Gym,Shopping Mall,Wine Shop,Italian Restaurant,Cupcake Shop,Food Truck,Fountain,9,5
4bc7ab4c14d79521c6e367e9,Battery Park City,40.711932,-74.016869,W New York - Downtown,40.709196,-74.013596,Hotel,Manhattan,40.711932,-74.016869,1,Park,Coffee Shop,Hotel,Gym,Shopping Mall,Wine Shop,Italian Restaurant,Cupcake Shop,Food Truck,Fountain,9,5
4a82eea3f964a52090f91fe3,Battery Park City,40.711932,-74.016869,New York Marriott Downtown,40.709521,-74.014486,Hotel,Manhattan,40.711932,-74.016869,1,Park,Coffee Shop,Hotel,Gym,Shopping Mall,Wine Shop,Italian Restaurant,Cupcake Shop,Food Truck,Fountain,9,5
544c4131498e767f7769b3e0,Battery Park City,40.711932,-74.016869,Holiday Inn Manhattan-Financial District,40.708286,-74.014084,Hotel,Manhattan,40.711932,-74.016869,1,Park,Coffee Shop,Hotel,Gym,Shopping Mall,Wine Shop,Italian Restaurant,Cupcake Shop,Food Truck,Fountain,9,5
56e6d85c498e04219119b385,Battery Park City,40.711932,-74.016869,Courtyard by Marriott New York Downtown Manhat...,40.709386,-74.012660,Hotel,Manhattan,40.711932,-74.016869,1,Park,Coffee Shop,Hotel,Gym,Shopping Mall,Wine Shop,Italian Restaurant,Cupcake Shop,Food Truck,Fountain,9,5
566c66b838fa37fd4827c98e,Financial District,40.707107,-74.010665,Q&A Residential Hotel,40.706556,-74.008143,Hotel,Manhattan,40.707107,-74.010665,1,Coffee Shop,Hotel,Gym,Wine Shop,Steakhouse,Pizza Place,Park,Bar,Food Truck,Italian Restaurant,22,5
57ab3eb5498e2efa275dfef3,Financial District,40.707107,-74.010665,AKA Wall Street,40.708030,-74.007850,Hotel,Manhattan,40.707107,-74.010665,1,Coffee Shop,Hotel,Gym,Wine Shop,Steakhouse,Pizza Place,Park,Bar,Food Truck,Italian Restaurant,22,5
54aed577498ee5fc196739f7,Financial District,40.707107,-74.010665,Residence Inn by Marriott New York Downtown Ma...,40.709605,-74.009827,Hotel,Manhattan,40.707107,-74.010665,1,Coffee Shop,Hotel,Gym,Wine Shop,Steakhouse,Pizza Place,Park,Bar,Food Truck,Italian Restaurant,22,5


#### Now we have all the hotels in each neighborhood, let's look at the ratings to find a good one

In [42]:
def gethotelrating(venue_id):
    
                  
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    
    result = requests.get(url).json()
    #print(result['response']['venue']['rating'])
    try:
        hotel_rating=result['response']['venue']['rating']
        #print(hotel_rating)
        
    except:
        hotel_rating='NA'
        #print(hotel_rating)
        
    # return only relevant information for each nearby venue
    return(hotel_rating)
    
    
    


In [48]:
#testing
#print(gethotelrating('4fe0634b7beb303528d17f94'))


In [45]:
df = []
for index, row in cls_hotels.iterrows():
    venue_id=row['Venue id']
    #print(venue_id)
    df.append(gethotelrating(venue_id))
   
    
df


[8.9,
 8.7,
 8.3,
 7.8,
 7.8,
 7.6,
 8.1,
 8.3,
 8.1,
 8.3,
 8.3,
 9.1,
 8.5,
 9.0,
 8.3,
 8.2,
 9.5]

In [46]:
cls_hotels['Hotel rating']=df

In [47]:
cls_hotels[['Neighborhood','Borough','Cluster Labels','restaurant_count','Hotel','Hotel rating']].sort_values('Hotel rating',ascending=False)

,Neighborhood,Borough,Cluster Labels,restaurant_count,Hotel,Hotel rating
1695,Midtown South,Manhattan,4,31,The NoMad Hotel,9.5
1602,Midtown South,Manhattan,4,31,Kimpton Hotel Eventi,9.1
1651,Midtown South,Manhattan,4,31,Ace Hotel New York,9.0
6,Chinatown,Manhattan,3,43,Hotel 50 Bowery NYC,8.9
1233,Battery Park City,Manhattan,1,9,Conrad New York,8.7
1648,Midtown South,Manhattan,4,31,MADE Hotel,8.5
1343,Financial District,Manhattan,1,22,AKA Wall Street,8.3
1379,Financial District,Manhattan,1,22,Andaz Wall Street,8.3
1388,Financial District,Manhattan,1,22,W New York - Downtown,8.3
1260,Battery Park City,Manhattan,1,9,W New York - Downtown,8.3


#### According to the results, we will choose the neighborhood ‘Midtown South’ with hotel ’The NoMad Hotel’ and neighborhood ‘Chinatown’ with hotel ‘Hotel 50 Bowery NYC’ to recommend to our client 
## Thank you!~